In [1]:
import sys
import os
import pandas as pandas
import numpy as numpy

from math import radians, cos, sin, asin, sqrt

from matplotlib import pyplot as plt
from scipy.spatial import distance
from sklearn.model_selection import train_test_split


from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR
from sklearn.model_selection import RandomizedSearchCV

In [2]:
#####################################################################################################################
######################### READING AND PREPROCESSING THE DATA ###########################################################




###########################################################

def read_data_from_csv_only_required_columns(directory_path, is_train_data):
    if is_train_data:
        required_columns = ['fare_amount', 'pickup_datetime', 'pickup_longitude', 'dropoff_longitude', 'pickup_latitude', 'dropoff_latitude', 'passenger_count']
        train_data = pandas.read_csv(directory_path, nrows = 500000, usecols = required_columns)
        return train_data
    else:
        test_data = pandas.read_csv(directory_path)
        return test_data




###########################################################

def convert_data_type_of_columns(data):
    data[['pickup_datetime']] = pandas.to_datetime(data['pickup_datetime'].str.slice(0, 19), utc = True,
                                                         format = '%Y-%m-%d %H:%M:%S')
    return data



###########################################################

def drop_null_values(data):
    data.dropna(inplace = True)
    return data
     
    
    
###########################################################

def remove_rows_with_fare_amount_less_than_zero(data):
    #print(numpy.shape(train_data))
    indices_of_rows_with_invalid_fare_amount = data.index[numpy.where(data['fare_amount'] < 2.50)]
    data.drop(indices_of_rows_with_invalid_fare_amount, inplace = True)
    return data

############################################################

'''city limits of NewYork'''
def get_and_remove_rows_with_gps_points_outside_the_limits(data):
    
    rows_with_pickup_longitude_outside_limits = get_indices_of_rows_with_gps_values_outside_limits(data,
                                                                                                 'pickup_latitude',
                                                                                                 40, 42)
    
    rows_with_pickup_latitude_outside_limits = get_indices_of_rows_with_gps_values_outside_limits(data,
                                                                                                 'pickup_longitude',
                                                                                                 -75, -72)
    
    rows_with_pickup_longitude_outside_limits = get_indices_of_rows_with_gps_values_outside_limits(data,
                                                                                                 'dropoff_latitude',
                                                                                                 40, 42)
    
    rows_with_pickup_longitude_outside_limits = get_indices_of_rows_with_gps_values_outside_limits(data,
                                                                                                 'dropoff_longitude',
                                                                                                 -75, -72)
    
    
    data = remove_rows_with_gps_points_outside_limits(data, rows_with_pickup_longitude_outside_limits)
    data = remove_rows_with_gps_points_outside_limits(data, rows_with_pickup_latitude_outside_limits)
    data = remove_rows_with_gps_points_outside_limits(data, rows_with_pickup_longitude_outside_limits)
    data = remove_rows_with_gps_points_outside_limits(data, rows_with_pickup_longitude_outside_limits)
    
    return data
    
    
    

    
############################################################

def get_indices_of_rows_with_gps_values_outside_limits(data, column_name, lower, higher):
    
    indices = data.index[numpy.where((data[column_name] < lower) &
                                                (data[column_name] > higher))]
    
    return indices

############################################################

def remove_rows_with_gps_points_outside_limits(data, row_indices):
    
    data.drop(row_indices, inplace = True)
    return data

    
    
###########################################################

'''passenger count should be in the limits of 1 and 6'''
def remove_rows_with_passenger_count_not_within_the_threshold(data):
    indices_of_rows_with_invalid_passenger_count = data.index[numpy.where((data['passenger_count'] < 1) | 
                                                                                (data['passenger_count'] > 6))]
    data.drop(indices_of_rows_with_invalid_passenger_count, inplace = True)
    return data



                    
###########################################################

# I got this  function from https://www.kaggle.com/alvaroibrain/my-attempt-to-nyc-taxi-fare-prediction.
def set_the_parameters_for_isInWater_func():
    #Custom mask made quickly with photoshop. Black pixels will be considered sea,
    newyork_img_mask = plt.imread('https://i.imgur.com/ov0cDqP.png')
    #Bounds of the area
    left_top = (-74.8, 41.1)
    right_bottom = (-72.8, 40.092)
    
    #Delta distance in limits
    distance_X = (right_bottom[0] - left_top[0])
    distance_Y = (left_top[1] - right_bottom[1])
    
    return newyork_img_mask, distance_X, distance_Y, left_top, right_bottom



###########################################################

# I got this  function from https://www.kaggle.com/alvaroibrain/my-attempt-to-nyc-taxi-fare-prediction.
def is_in_water(ny_img_mask, left_top, right_bottom, distance_X, distance_Y, coord):
    
    width = ny_img_mask.shape[1]
    height = ny_img_mask.shape[0]
    array_sea_color = numpy.array([0., 0. , 0.], dtype='float32')
    
    #Map delta distance to pixels
    pix_x = ((coord[0] - left_top[0]) / distance_X) * width
    pix_y = ((coord[1] - right_bottom[1]) / distance_Y) * height

    if pix_x < 0 or pix_y < 0 or pix_x > width or pix_y > height:
        return True #coord outside bounds
    
    #Get the color of the pixel
    color = ny_img_mask[ height- int(pix_y), int(pix_x)]
    
    #Is in sea?  (compare color)
    return numpy.array_equal(color, array_sea_color)




###########################################################

def get_indices_of_rows_where_coordinates_are_not_on_land(data, ny_img_mask, distance_X, distance_Y, left_top, right_bottom):
        
    bool_values_of_pickup_coordinates_not_on_land = data.apply(lambda row: is_in_water(ny_img_mask, left_top, right_bottom, 
                                                                            distance_X, distance_Y,
                                                                            (row.pickup_longitude, 
                                                                             row.pickup_latitude)), axis = 1)
    
    bool_values_of_dropoff_coordinates_not_on_land = data.apply(lambda row: is_in_water(ny_img_mask, left_top, right_bottom, 
                                                                            distance_X, distance_Y,
                                                                            (row.dropoff_longitude, 
                                                                             row.dropoff_latitude)), axis = 1)
    

    indices_of_rows_pickup_coordinates_not_on_land = list(data.index[numpy.where(bool_values_of_pickup_coordinates_not_on_land)])

    indices_of_rows_drop_coordinates_not_on_land   = list(data.index[numpy.where(bool_values_of_dropoff_coordinates_not_on_land)])
    
    indices_of_rows_with_coordinates_not_on_land = list(set().union(indices_of_rows_pickup_coordinates_not_on_land,
                                                             indices_of_rows_drop_coordinates_not_on_land))

      
    return numpy.array(indices_of_rows_with_coordinates_not_on_land)




###########################################################

def remove_rows_with_coordinates_not_on_land(data):
    
    newyork_img_mask, distance_X, distance_Y, left_top, right_bottom = set_the_parameters_for_isInWater_func()
    indices_of_rows_with_coordinates_not_on_land = get_indices_of_rows_where_coordinates_are_not_on_land(data, newyork_img_mask, 
                                                                                                         distance_X, distance_Y, 
                                                                                                         left_top, right_bottom )
    
    data.drop(indices_of_rows_with_coordinates_not_on_land, inplace = True)
    
    
    return data


############################################################

def get_the_target_variable_and_remove_useless_columns(data, is_train_data):
    
    if is_train_data:
        train_Y = data['fare_amount']
        train_data = data.drop(['fare_amount', 'pickup_datetime'], axis = 1)
        return train_data, train_Y
    
    else:
        data = data.drop(['key', 'pickup_datetime'], axis = 1)
        return data
    


    
###########################################################

def preprocess_data(data, is_train_data):
    data = convert_data_type_of_columns(data)
    data = drop_null_values(data)
    if is_train_data:
        data = remove_rows_with_fare_amount_less_than_zero(data)
        
    data = remove_rows_with_passenger_count_not_within_the_threshold(data)
    data = remove_rows_with_coordinates_not_on_land(data)
    data = get_and_remove_rows_with_gps_points_outside_the_limits(data)
    return data





                                                

In [3]:


#####################################################################################################################
######################### ADDING MORE FEATURES TO THE DATA ###########################################################


def get_absolute_diff_between_pickup_dropoff_values(pickup, dropoff):
    return abs((pickup - dropoff))



###########################################################

'''x and y are tuples of length 2'''
def calculate_euclidean_distance(x, y):
    return numpy.sqrt(((x[0] - y[0]) ** 2) + ((x[1] - y[1]) ** 2))




###########################################################

def calculate_manhattan_distance(x, y):
    return ((y[0] - x[0]).abs() + (y[1] - x[1]).abs())



###########################################################

'''got this from here 
https://stackoverflow.com/questions/15736995/how-can-i-quickly-estimate-the-distance-between-two-latitude-longitude-points'''
def calculate_haversine_distance(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km


###########################################################

def get_the_hour_of_booking(booking_datetime):
    return booking_datetime.hour



###########################################################

def get_the_day_of_booking(booking_datetime):
    return booking_datetime.day


###########################################################

def add_columns_to_data(data):

    data['euclidean_dist'] = calculate_euclidean_distance((data['pickup_longitude'], data['pickup_latitude']),
                                (data['dropoff_longitude'], data['dropoff_latitude'])).astype(numpy.float32)

    data['manhattan_dist'] = calculate_manhattan_distance((data['pickup_longitude'], data['pickup_latitude']),
                                (data['dropoff_longitude'], data['dropoff_latitude'])).astype(numpy.float32)
        
    data['haversine_distance'] = data.apply(lambda row: calculate_haversine_distance(row.pickup_longitude, 
                                                                                                 row.pickup_latitude, 
                                                                                                 row.dropoff_longitude, 
                                                                                                 row.dropoff_latitude), 
                                                                                                            axis = 1)
    
    data['abs_longitude_diff'] = data.apply(lambda row: get_absolute_diff_between_pickup_dropoff_values(
                                                                    row.pickup_longitude, row.dropoff_longitude), 
                                                                        axis = 1)
    
    data['abs_latitude_diff'] = data.apply(lambda row: get_absolute_diff_between_pickup_dropoff_values(
                                                                    row.pickup_latitude, row.dropoff_latitude), 
                                                                        axis = 1)
    return data



# ###########################################################

def find_correlation_between_columns_of_data(data):
    corr_euclideanDist_fareAmount = data['euclidean_dist'].corr(data['fare_amount'])
    corr_manhattanDist_fareAmount = data['manhattan_dist'].corr(data['fare_amount'])
    #corr_hourOfBooking_fareAmount = data['hour_of_booking'].corr(data['fare_amount'])
    corr_abs_long_diff_fareAmount = data['abs_longitude_diff'].corr(data['fare_amount'])
    corr_abs_lat_diff_fareAmount = data['abs_latitude_diff'].corr(data['fare_amount'])
    
    print('Euclidean - fare - ' + str(corr_euclideanDist_fareAmount))
    print('Manhattan - fare - ' + str(corr_manhattanDist_fareAmount))
    print('Euclidean - fare - ' + str(corr_euclideanDist_fareAmount))
    print('abs_long - fare - ' + str(corr_abs_long_diff_fareAmount))
    print('abs_lat - fare - ' + str(corr_abs_lat_diff_fareAmount))
    #print('Hour - fare - ' + str(corr_hourOfBooking_fareAmount))
    

                                                                                                               

In [4]:

#####################################################################################################################
######################### STANDARDIZING THE DATA ###########################################################


def standardize_the_data(data):
    scaler = StandardScaler()
    data_standardized = scaler.fit_transform(data)
    return data_standardized

In [5]:


########################################################################################################
################################### LINEAR  REGRESSION #################################################

def linear_regressor(train_X, train_Y):
    train_X_standardized = standardize_the_data(train_X)
    regressor = LinearRegression().fit(train_X_standardized, (numpy.array(train_Y).reshape(-1, 1)))    
    return regressor



In [6]:

########################################################################################################
################################### RIDGE  REGRESSION ##################################################

def perform_ridgeCV_for_best_alpha_and_predicting(train_X, train_Y):
    
    ridgeCV = RidgeCV(alphas = [0.03, 0.05, 0.1, 0.5, 1, 5, 10])
    best_alpha_value = ridgeCV.fit(train_X, (numpy.array(train_Y).reshape(-1, 1)))
    print('the best alpha is ' + str(best_alpha_value.alpha_))
    return ridgeCV



def perform_ridge_regression_on_the_data(train_X, train_Y):
    train_X_standardized = standardize_the_data(train_X)
    
    ridgeCV_classifier = perform_ridgeCV_for_best_alpha_and_predicting(train_X_standardized, train_Y)
    
    
    return ridgeCV_classifier


In [7]:

########################################################################################################
################################### RANDOM SEARCH AND GENERIC REGRESSION ##########################################



def random_grid_search(train_X, train_Y, search_grid_params_dict, regressor):

    
    random_search_grid = RandomizedSearchCV(estimator = regressor, param_distributions = search_grid_params_dict,
                                              n_iter = 5, cv = 2, verbose = 2, random_state = 23, n_jobs = -1)
        
    random_search_grid.fit(train_X, train_Y)
    best_params = random_search_grid.best_params_
    
    
    return best_params



############################################################

def perform_regression(regressor, train_X, train_Y):
    
    regressor.fit(train_X, train_Y)
    return regressor

In [8]:

########################################################################################################
################################### SUPPORT VECTOR REGRESSION ##########################################



def set_params_for_grid_search():
    
    epsilon_value = [0.3, 0.5, 0.9]
    C_value = [(pow(2, exponent)) for exponent in range(-3, 0)]
    tolerance = [(pow(2, exponent)) for exponent in range(-10, -8)]
    search_grid_params = {'epsilon' : epsilon_value,
                          'C' : C_value,
                          'tol' : tolerance
                         }
    return search_grid_params




        
def perform_regression_on_data_using_SVR(train_X, train_Y):
    
    train_X_standardized = standardize_the_data(train_X)

    SVR_regressor = LinearSVR()
    
    set_params_for_grid_search_SVR = set_params_for_grid_search()
    best_params_dict = random_grid_search(train_X, train_Y, set_params_for_grid_search_SVR, SVR_regressor)
    
    print('best params for svm are ')
    print(best_params_dict)
    
    
    epsilon_value = best_params_dict['epsilon']
    C_value = best_params_dict['C']
    tolerance_value = best_params_dict['tol']

    
    regressor = LinearSVR(epsilon = epsilon_value, tol = tolerance_value, C = C_value, loss = 'squared_epsilon_insensitive',
                                                       dual = False, random_state = 42)
    regressor  = perform_regression(regressor, train_X_standardized, train_Y)
    
    
    return regressor





In [9]:

########################################################################################################
################################### STOCHASTIC GRADIENT DESCENT ##########################################

from sklearn.linear_model import SGDRegressor

def set_params_for_grid_search_SGD():
    
    loss_func = ['squared_loss', 'huber']
    epsilon_value = [0.3, 0.5, 0.9]
    alpha_value = [pow(10, exponent) for exponent in range(-4, -1)]
    rate_of_learning = ['invscaling', 'adaptive']
    
    search_grid_params = {'loss' : loss_func,
                          'alpha': alpha_value,
                          'epsilon' : epsilon_value,
                          'learning_rate' : rate_of_learning
                         }
    return search_grid_params



# ###########################################################

def perform_regression_on_data_using_SGD(train_X, train_Y):
    
    
    train_X_standardized = standardize_the_data(train_X)
    
    SGD_regressor = SGDRegressor()
    
    params_for_grid_search_SGD = set_params_for_grid_search_SGD()
    best_params_dict = random_grid_search(train_X_standardized, train_Y, 
                                                  params_for_grid_search_SGD, SGD_regressor)
    
    print('best params for SGD are')
    print(best_params_dict)
    
    
    loss_func = best_params_dict['loss']
    epsilon_value = best_params_dict['epsilon']
    rate_of_learning = best_params_dict['learning_rate']
    alpha_value = best_params_dict['alpha']


    
    regressor = SGDRegressor(loss = loss_func, epsilon = epsilon_value,  learning_rate = rate_of_learning, 
                                                                                             random_state = 42)
    
    regressor = perform_regression(regressor, train_X_standardized, train_Y)
    
    
    return regressor






In [10]:
train_data = read_data_from_csv_only_required_columns('/train.csv', True)
test_data = read_data_from_csv_only_required_columns('/test.csv', False)
print(train_data.info())
print("-----------------")
print(test_data.info())


train_data = preprocess_data(train_data, True)
test_data = preprocess_data(test_data, False)
print("*************************************")
print(train_data.info())
print("-----------------")
print(test_data.info())




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 7 columns):
fare_amount          500000 non-null float64
pickup_datetime      500000 non-null object
pickup_longitude     500000 non-null float64
pickup_latitude      500000 non-null float64
dropoff_longitude    499995 non-null float64
dropoff_latitude     499995 non-null float64
passenger_count      500000 non-null int64
dtypes: float64(5), int64(1), object(1)
memory usage: 26.7+ MB
None
-----------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9914 entries, 0 to 9913
Data columns (total 7 columns):
key                  9914 non-null object
pickup_datetime      9914 non-null object
pickup_longitude     9914 non-null float64
pickup_latitude      9914 non-null float64
dropoff_longitude    9914 non-null float64
dropoff_latitude     9914 non-null float64
passenger_count      9914 non-null int64
dtypes: float64(4), int64(1), object(2)
memory usage: 542.2+ KB
None
********************

In [11]:
train_data = add_columns_to_data(train_data)
test_data = add_columns_to_data(test_data)


In [12]:
train_data_X, train_data_Y   = get_the_target_variable_and_remove_useless_columns(train_data, True)



In [13]:
linear_regressor_ = linear_regressor(train_data_X,  train_data_Y)
                                                                                           



/anaconda3/envs/tensorflow-session/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype float32, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/envs/tensorflow-session/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype float32, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [14]:
ridge_regressor = perform_ridge_regression_on_the_data(train_data_X,  train_data_Y)
                                                                                                  

/anaconda3/envs/tensorflow-session/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype float32, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/envs/tensorflow-session/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype float32, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


the best alpha is 0.1


In [22]:
SVR_regressor = perform_regression_on_data_using_SVR(train_data_X, train_data_Y)

/anaconda3/envs/tensorflow-session/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype float32, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/envs/tensorflow-session/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype float32, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 2 folds for each of 5 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  4.3min finished
/anaconda3/envs/tensorflow-session/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


best params for svm are 
{'tol': 0.0009765625, 'epsilon': 0.3, 'C': 0.125}


In [15]:
SGD_regressor = perform_regression_on_data_using_SGD(train_data_X, train_data_Y)

Fitting 2 folds for each of 5 candidates, totalling 10 fits


/anaconda3/envs/tensorflow-session/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype float32, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/envs/tensorflow-session/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype float32, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    5.8s finished
/anaconda3/envs/tensorflow-session/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 10

best params for SGD are
{'loss': 'squared_loss', 'learning_rate': 'invscaling', 'epsilon': 0.9, 'alpha': 0.001}


/anaconda3/envs/tensorflow-session/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


In [16]:
test_key = test_data['key']
test_data = get_the_target_variable_and_remove_useless_columns(test_data, False)
test_data_standardized = standardize_the_data(test_data)

/anaconda3/envs/tensorflow-session/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype float32, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/envs/tensorflow-session/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype float32, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [17]:
SGD_predicted_Y = SGD_regressor.predict(test_data_standardized)

In [23]:
SVR_predicted_Y = SVR_regressor.predict(test_data_standardized)

In [18]:
ridge_predicted_Y = ridge_regressor.predict(test_data_standardized)
ridge_predicted_Y_list = [y[0] for y in ridge_predicted_Y]

In [19]:
linear_predicted_Y = linear_regressor_.predict(test_data_standardized)
linear_predicted_Y_list = [y[0] for y in linear_predicted_Y]

In [20]:
def get_the_submission_file(predicted_Y, key, regressor_name):
    df = pandas.DataFrame({'key' : key, 'fare_amount' : (predicted_Y)}, columns = ['key', 'fare_amount'])
    return df.to_csv((regressor_name + '_submissions.csv'), index = False)


In [24]:
linear_submission = get_the_submission_file(list(linear_predicted_Y_list), list(test_key), 'linear')
ridge_submission = get_the_submission_file(list(ridge_predicted_Y_list), list(test_key), 'ridge')
SVR_submission = get_the_submission_file(list(SVR_predicted_Y), list(test_key), 'SVR')
SGD_submission = get_the_submission_file(list(SGD_predicted_Y), list(test_key), 'SGD')